In [99]:
include("utils/data_loader.jl")
include("utils/visualization.jl")
include("utils/preprocessing.jl")

Main.Preprocessing

## APPROACH 1: Without Balancing ##

In [100]:
# import Pkg;
# Pkg.add("Random")
using Random
Random.seed!(123)

data = DataLoader.load_data("dataset\\star_classification.csv");

##### Preprocess and balancing data #####

In [101]:
# preprocess_data(dataset, balancing_dataset, normalization, features)
inputs, targets = Preprocessing.preprocess_data(data, false, [4,5,6,7,8]);

# Print first input and target
println("First input: ", inputs[1, :])
println("First target: ", targets[1, :])

Size :(100000, 5)
First input: Float32[23.87882, 22.2753, 20.39501, 19.16573, 18.79371]
First target: Bool[1, 0, 0]


In [102]:
# Split to train and test
# train_inputs, train_targets, test_inputs, test_targets = DataLoader.split_data(inputs, targets, 0.8);
# Check size of train and test sets
# println("Train inputs: ", size(train_inputs))
# println("Train targets: ", size(train_targets))
# println("Test inputs: ", size(test_inputs))
# println("Test targets: ", size(test_targets))

In [103]:
# Normalize data
train_inputs = Preprocessing.normalize_data(inputs , "minmax");
println("Train inputs: ", train_inputs[1, :])
println("Train targets: ", train_targets[1, :])

Train inputs: Float32[0.9991126, 0.9990701, 0.5353444, 0.42766455, 0.998944]
Train targets: Bool[1, 0, 0]


In [104]:
println("Train inputs: ", size(train_targets))
training_indices = Preprocessing.crossvalidation(targets, 10)
println("Crossvalidation indices: ", training_indices[1:100])

Train inputs: (80000, 3)
Crossvalidation indices: [10, 1, 7, 4, 9, 6, 10, 6, 6, 5, 8, 1, 5, 9, 1, 7, 9, 8, 9, 9, 1, 6, 1, 6, 3, 10, 5, 2, 9, 8, 5, 4, 1, 2, 7, 9, 1, 8, 5, 7, 5, 2, 6, 2, 5, 9, 7, 9, 1, 4, 8, 7, 2, 9, 4, 4, 1, 7, 2, 7, 2, 5, 9, 9, 8, 8, 9, 7, 6, 5, 8, 1, 8, 6, 10, 8, 8, 2, 10, 6, 9, 5, 6, 10, 1, 8, 9, 8, 9, 9, 3, 4, 1, 5, 2, 10, 10, 1, 10, 9]


In [107]:
using ScikitLearn

@sk_import svm: SVC
@sk_import tree: DecisionTreeClassifier
@sk_import neighbors: KNeighborsClassifier

function train_model(estimators::AbstractArray{Symbol,1},
                    modelsHyperParameters::AbstractArray{Dict{String,Any},1},
                    trainingDataset::Tuple{Matrix{Float32}, BitMatrix},
                    kFoldIndices::Array{Int64,1})
    
    numFolds = maximum(kFoldIndices)
    (inputs, targets) = trainingDataset

    testAccuracies = Array{Float64}(undef, numFolds);

    for numFold in 1:numFolds
        trainingInputs = inputs[kFoldIndices .!= numFold, :];
        testingInputs = inputs[kFoldIndices .== numFold, :];
        trainingTargets = targets[kFoldIndices .!= numFold, :];
        testingTargets = targets[kFoldIndices .== numFold, :];
        
        #Define the models to train
        models = Dict("SVM" => SVC(probability=modelsHyperParameters[1]["probability"], kernel=modelsHyperParameters[1]["kernel"], C=modelsHyperParameters[1]["C"]), 
                "DT" => DecisionTreeClassifier(max_depth=modelsHyperParameters[2]["max_depth"]),
                "KNN" => KNeighborsClassifier(n_neighbors=modelsHyperParameters[3]["n_neighbors"]))

        base_models = [name for name in keys(models)]

        for key in keys(models)
            model = models[key]
            fit!(model,trainingInputs, trainingTargets)

            acc = score(model,testInputs, testTargets)
            println("$key: $(acc*100) %")
        end
    end
end

train_model (generic function with 3 methods)

In [109]:
modelsHyperParameters = Array{Dict{String, Any}}(undef, 3)
modelsHyperParameters[1] = Dict("probability" => true, "kernel" => "linear", "C" => 0.1)
modelsHyperParameters[2] = Dict("max_depth" => 4)
modelsHyperParameters[3] = Dict("n_neighbors" => 3)

ensemble_accuracy = train_model([:SVM, :DT, :KNN], modelsHyperParameters, (inputs, targets), training_indices)

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\marci\.julia\packages\PyCall\ilqDX\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('y should be a 1d array, got an array of shape (89998, 3) instead.')
  File "C:\Users\marci\.julia\conda\3\x86_64\lib\site-packages\sklearn\svm\_base.py", line 192, in fit
    X, y = self._validate_data(
  File "C:\Users\marci\.julia\conda\3\x86_64\lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\marci\.julia\conda\3\x86_64\lib\site-packages\sklearn\utils\validation.py", line 1122, in check_X_y
    y = _check_y(y, multi_output=multi_output, y_numeric=y_numeric, estimator=estimator)
  File "C:\Users\marci\.julia\conda\3\x86_64\lib\site-packages\sklearn\utils\validation.py", line 1143, in _check_y
    y = column_or_1d(y, warn=True)
  File "C:\Users\marci\.julia\conda\3\x86_64\lib\site-packages\skl